Próba implementacji metod zastosowanych przez dr. Kontka w artykule 

["**Weryfikacja wyniku drugiej tury wyborów prezydenckich w Polsce w 2025 roku: Przeliczenie głosów z użyciem przestrzennie grupowanej metody MAD**"](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=5296441)

wraz z komentarzami i wątpliwościami

In [1]:
import pandas as pd

from utilities import presidential_data, clustering, methods

### load presidential both rouds - official PKW data

In [2]:
year = "2025"
df_2025_r1 = presidential_data.get_df(year, "1")
df_2025_r2 = presidential_data.get_df(year, "2")

### 2.2. Grupowanie geograficzne

z artykułu:

W drugim etapie komisje wyborcze zostały posortowane według kodów pocztowych, a następnie
pogrupowane w kolejne bloki komisji znajdujących się w bezpośrednim sąsiedztwie. Grupy tworzono
w taki sposób, aby — w miarę możliwości — każda zawierała od 10 do 16 komisji, łącząc ze sobą
sąsiednie obszary kodów pocztowych mające wspólny prefiks (np. „30”, „301”, „3011”). Celem było
maksymalne zwiększenie spójności przestrzennej przy zachowaniu poręcznej wielkości grupy, bez
konieczności ograniczania jej do jednego kodu pocztowego. Zastosowano następującą procedurę:

  1. Początkowe grupowanie oparto na pierwszych dwóch cyfrach kodu pocztowego (np. „30” dla
obszaru Krakowa).
  2. Jeżeli powstała grupa zawierała od 10 do 16 komisji, została zaakceptowana bez zmian.
  3. Grupy liczące mniej niż 10 komisji odłożono do późniejszego łączenia.
  4. Grupy przekraczające 16 komisji dzielono rekurencyjnie, dodając kolejne cyfry kodu pocztowego (np. z „30” → „301” → „3011” i dalej, aż do pełnych pięciu cyfr).
  5. Pozostałe małe grupy łączono z najbliższymi sąsiadami mającymi ten sam prefiks, przy czym priorytetem była ciągłość przestrzenna i zrównoważona liczebność grup.
  
W odróżnieniu od wcześniejszego podejścia, które dopuszczało grupy o wielkości 10–25 komisji,
niniejsze badanie przyjęło węższy zakres docelowy: od 10 do 16 komisji na grupę. Decyzja ta
wynikała z przeglądu empirycznego, który wykazał, że większe grupy — mimo wydajności
statystycznej — czasami łączyły odległe geograficznie obszary o niejednorodnych wzorcach
głosowania.

W wyniku zastosowania nowych ograniczeń utworzono 2 208 grup, z których każda odzwierciedlała
względnie jednorodną lokalną dynamikę wyborczą. Dla potwierdzenia ich spójności terytorialnej
przeprowadzono test zgodności kodów pocztowych w ramach każdej grupy.

Większość grup spełniła założony docelowy rozmiar: 1 386 grup (62,8%) zawierało od 10 do 16
komisji, a 2 017 grup (91,3%) zawierało od 6 do 30 komisji. Większe grupy zazwyczaj odpowiadały obszarom miejskim — na przykład takim jak Toruń czy Włocławek, gdzie pojedynczy kod pocztowy
obejmował całe miasto. W takich przypadkach większa liczba komisji nie zaburzała spójności
przestrzennej, a wręcz zwiększała wiarygodność statystyczną poprzez powiększenie próbki lokalnej.

Grupy mniejsze niż docelowy zakres obejmowały komisje, które — z powodu izolacji geograficznej
— nie mogły zostać sensownie połączone z innymi. Choć próbki mniejsze niż 10 jednostek są zwykle
uznawane za mające ograniczoną moc statystyczną, zastosowanie metody MAD — znanej ze swojej
odporności na małe próby — w znacznym stopniu niweluje to ograniczenie.

------------

Nie jestem w stanie odtworzyć grupowania geograficznego 1:1.

Jedną z opcji jest grupowanie losowe: buckety 10 - 16, dokladnie 2208 grup

In [3]:
df = clustering.add_random_buckets(df_2025_r2, n=2208, k_min=10, k_max=16)

In [4]:
clustering.print_bucket_stats(df)

Total buckets: 2208
Buckets with 10–16 items: 2208 (100.0%)
Buckets with 6–30 items: 2208 (100.0%)


Drugą moje "klastrowanie"

In [5]:
df = clustering.add_janiszewski_postal_buckets(df_2025_r2, min_bucket_size=10, max_bucket_size=16)

In [6]:
clustering.print_bucket_stats(df)

Total buckets: 1809
Buckets with 10–16 items: 1462 (80.8%)
Buckets with 6–30 items: 1719 (95.0%)


Trzecią klastrowanie Jakuba Bialka

https://github.com/rabitwhte/analiza_kontka_reprodukcja/blob/main/Reprodukcja_wynikow_Kontek_Bialek.ipynb

In [7]:
df = clustering.add_bialek_postal_buckets(df_2025_r2, min_size=10, max_size=16)

In [8]:
clustering.print_bucket_stats(df)

Total buckets: 2367
Buckets with 10–16 items: 1223 (51.7%)
Buckets with 6–30 items: 1926 (81.4%)


### IMPLEMENTACJA METOD ZAPROPONOWANYCH PRZEZ DR KONTKA

2.3. Wykrywanie wartości odstających

Główna innowacja analityczna niniejszego badania polega na oszacowaniu potencjalnego wpływu
anormalnych komisji wyborczych na poziomie ogólnokrajowym. Aby to osiągnąć, w pierwszej
kolejności zidentyfikowano wartości odstające w czterech kategoriach nieprawidłowości:

In [9]:
cand_A = "trzaskowski"
cand_B = "nawrocki"

df_2025 = presidential_data.join_both_rounds(cand_A, cand_B, df_2025_r1, df_2025_r2)

In [10]:
# # ok, grupowanie metodą p. Białka
df_2025 = clustering.add_bialek_postal_buckets(df_2025)

### 1. pop_outlier: Nadmierne poparcie dla Karola Nawrockiego (względem mediany w ramach lokalnej grupy)

Za artykułem: w ramach każdej grupy obliczono mediany oraz odchylenia bezwzględne od mediany (MAD), dla każdej komisji obliczono wskaźnik odchylenia od mediany

![X minus median over MAD](./../../images/X_minus_median_over_MAD.png)

gdzie:

X - wynik w drugiej turze kandydata

mediana - mediana wyników kandydata w drugiej turze

MAD - odchelenie bezwzględne mediany kandydata w drugiej turze

z artykulu:

Dla każdej grupy komisji oraz dla dwóch pierwszych kategorii nieprawidłowości:

  • obliczono mediany oraz odchylenia bezwzględne od mediany (MAD);

  • dla każdej komisji obliczono tzw. współczynnik odchylenia odpornego (robust deviation score, oznaczony jako k_needed), wyrażający skalę odchylenia wyniku od mediany grupy w jednostkach MAD, według wzoru (powyzej)

Komisję oznaczano jako odstającą (outlier), jeśli spełniony był warunek: 𝑘𝑛𝑒𝑒𝑑𝑒𝑑 > 𝑘 gdzie k to
wartość progowa przyjęta w analizie, szczegółowo opisana w sekcji 2.4.

In [11]:
print("Liczba komisji: ", len(df_2025)) # bez zagranicy i statków
df = df_2025.copy()

Liczba komisji:  30553


2.4 Przeliczenie wyników

Aby uwzględnić niepewność i wrażliwość zastosowanego podejścia, obliczenia przeprowadzono dla
trzech różnych progów detekcji wartości odstających: k > 2.0, k > 2.5 oraz k > 3.0, gdzie k oznacza
liczbę jednostek odchylenia bezwzględnego od mediany (MAD) względem mediany w grupie
lokalnej. Wyższe wartości k wyodrębniają jedynie najbardziej skrajne przypadki, zapewniając tym
samym konserwatywną estymację potencjalnego wpływu. Jednocześnie jednak ograniczają zdolność
metody do wychwytywania mniejszych, lecz wciąż istotnych odchyleń.

In [12]:
df = methods.add_anomaly_1(df, cand_A, cand_B, new_col_name="k_score_1")

In [13]:
methods.check_k_thresholds(df, cand_A, cand_B, "_k_score_1")

k > 2.0
trzaskowski: 4551
nawrocki: 3750
---
k > 2.5
trzaskowski: 3535
nawrocki: 2750
---
k > 3.0
trzaskowski: 2794
nawrocki: 2015
---


Wyniki:

Dla k=2, takich komisji, w których "za duże" poparcie ma Nawrocki jest 3750, a Trzaskowski 4551.

### 2. growth_outlier: Nadmierny względny wzrost poparcia dla Karola Nawrockiego między pierwszą a drugą turą, w porównaniu do odpowiedniego wzrostu poparcia dla Rafała Trzaskowskiego w tej samej grupie lokalnej;

za, JB: 

Nie podano wprost jak to było obliczone więc kolejno:

  1. Dla danego kandydata obliczam względny wzrost między pierwszą a drugą turą (dzieląc wynik z drugiej przez wynik z pierwszej)
  2. Następnie odnoszę go do wzrostu drugiego kandydata - liczę różnicę między względnymi wzrostami.
  3. Dalej tak jak w pierwszym typie anomalii - dla tych różnic liczę medianę grupy, MAD grupy oraz odchylenie k w komisji.

In [14]:
df = methods.add_anomaly_2(df, cand_A, cand_B, new_col_name="k_score_2")

In [15]:
df.head()

,teryt_gmina,polling_station_id,trzaskowski_r1,nawrocki_r1,postal_code,trzaskowski_r2,nawrocki_r2,postal_clean,bucket,trzaskowski_k_score_1,nawrocki_k_score_1,trzaskowski_k_score_2,nawrocki_k_score_2
0,20101,1,361,287,59-700,596,582,59700,1567,1.191686,2.942708,-0.269043,0.269043
1,20101,2,381,228,59-700,517,386,59700,1567,0.826790,0.901042,-0.022318,0.022318
2,20101,3,356,241,59-700,556,437,59700,1567,1.006928,1.432292,0.487976,-0.487976
3,20101,4,390,217,59-700,567,408,59700,1567,1.057737,1.130208,-0.567403,0.567403
4,20101,5,343,202,59-700,518,332,59700,1567,0.831409,0.338542,1.200906,-1.200906


In [16]:
methods.check_k_thresholds(df, cand_A, cand_B, "_k_score_2")

k > 2.0
trzaskowski: 3552
nawrocki: 3127
---
k > 2.5
trzaskowski: 2666
nawrocki: 2229
---
k > 3.0
trzaskowski: 2106
nawrocki: 1669
---


### 3. Komisje, w których Nawrocki uzyskał więcej głosów niż Trzaskowski w drugiej turze, mimo że mediana wyników w grupie wskazywała na przewagę Trzaskowskiego;

  1. Sprawdzamy, w których grupach dany kandydat miał większą medianę
  2. Sumujemy komisje, w których wygrał kandydat A mimo, że większą medianę miał kandydat B i na odwrót.

In [17]:
df = methods.add_anomaly_3(df, cand_A, cand_B, new_col_name="flip")

In [18]:
# na korzyść kandydat A, czyli większą medianę miał B, a więcej głosów dostał A.
cand_A, sum(df['higher_median_' + cand_B] & (df[cand_A + '_r2'] > df[cand_B + '_r2']))

('trzaskowski', 2608)

In [19]:
cand_B, sum(df['higher_median_' + cand_A] & (df[cand_B + '_r2'] > df[cand_A + '_r2']))

('nawrocki', 1843)

In [20]:
df = df.drop(
    columns=[
        f"higher_median_{cand_A}",
        f"higher_median_{cand_B}",
    ]
)

**WYNIKI**:

W grupach, w których większą medianę miał Nawrocki, było 2608 komisji, w których wyższy wynik uzyskał Trzaskowski.

W grupach, w których większą medianę miał Trzaskowski, było 1843 komisji, w których wyższy wyniki uzyskał Nawrocki.

Przykładowo:

W komisji 13 gdzie w drugiej turze głosowało.. 13 osób, Trzaskowski uzyskał większy wynik (8 do 5), mimo że w grupie obejmującej kod pocztowy 59-730 większą medianę miał Nawrocki (344 vs 158).

**Wątpliwości w tej metodologii**

Jak pisze Piotr Szulc:

https://danetyka.com/kontek-analiza-bledow/

Jedna z cech, jakie bada autor, jest nazwana “flip” i nie ma nic wspólnego z wyżej podaną standaryzacją i progami. Autor za anomalię uznaje każdy przypadek, w którym “Nawrocki wygrywa lokalnie, mimo że mediana wyników w grupie wskazuje przewagę Trzaskowskiego”. Załóżmy, że procenty poparcia dla Nawrockiego w danej grupie wynoszą: 45, 46, 47, 48, 49, 51, 52, 53, 54.

In [21]:
# Dane: 9 komisji – Trzaskowski ma wyższą medianę, ale Nawrocki wygrywa w 4 komisjach
dummy_df = pd.DataFrame({
    'okręg': ['A'] * 9,
    'trzaskowski': [55, 54, 53, 52, 51, 49, 47, 46, 45],
    'nawrocki':    [45, 46, 47, 48, 49, 51, 52, 53, 54],
})

Mediana wynosi 49%, więc “mediana wyników w grupie wskazuje przewagę Trzaskowskiego”:

In [22]:
# Obliczenie median
trzaskowski_median = dummy_df['trzaskowski'].median()  # 51.0
nawrocki_median = dummy_df['nawrocki'].median()        # 49.0

# ale zeby być spójnym z poprzednią implementacją:

# mediana w grupie
dummy_df[cand_A + '_median'] = dummy_df.groupby('okręg')[cand_A].transform('median')
dummy_df[cand_B + '_median'] = dummy_df.groupby('okręg')[cand_B].transform('median')

dummy_df['higher_median_' + cand_A] = (dummy_df[cand_A + '_median'] >  dummy_df[cand_B + '_median']).astype(bool)
dummy_df['higher_median_' + cand_B] = (dummy_df[cand_B + '_median'] >  dummy_df[cand_A + '_median']).astype(bool)

a zatem te cztery komisje, w których Nawrockich otrzymał ponad 50% to anomalie, co oczywiście nie ma żadnego sensu. Ta cecha jest odpowiedzialna za ponad połowę (!) wskazań.

In [23]:
# na korzyść kandydata B, czyli większą medianę miał A, a więcej głosów dostał B.
cand_B, sum(dummy_df['higher_median_' + cand_A] & (dummy_df[cand_B] > dummy_df[cand_A]))

('nawrocki', 4)

In [24]:
# Flip: Nawrocki wygrywa, mimo że Trzaskowski miał wyższą medianę w grupie
dummy_df['flip_' + cand_B] = dummy_df['higher_median_' + cand_A] & (dummy_df[cand_B] > dummy_df[cand_A])

# Wyświetlenie flipów
print(dummy_df[[cand_A, cand_B, 'flip_' + cand_B]])
print(f"\nLiczba 'anomalii' według flip: {dummy_df['flip_' + cand_B].sum()} z {len(dummy_df)}")

   trzaskowski  nawrocki  flip_nawrocki
0           55        45          False
1           54        46          False
2           53        47          False
3           52        48          False
4           51        49          False
5           49        51           True
6           47        52           True
7           46        53           True
8           45        54           True

Liczba 'anomalii' według flip: 4 z 9


### 4. Kandydat otrzymał mniej głosów w drugiej turze niż w pierwszej

In [25]:
cand_A, sum(df[cand_A + '_r2']<df[cand_A + '_r1'])

('trzaskowski', 128)

W 128 komisjach Trzaskowski uzyskał mniej głosów w drugiej turze niż w pierwszej.

In [26]:
cand_B, sum(df[cand_B + '_r2']<df[cand_B + '_r1'])

('nawrocki', 112)

W 112 komisjach Nawrocki uzyskał mniej głosów w drugiej turze niż w pierwszej.


Przykładowe anomalie na korzyść Trzaskowskiego:

In [27]:
df[df[cand_B + '_r2'] < df[cand_B + '_r1']].sort_values(by=cand_B + '_r1', ascending=False).head()

,teryt_gmina,polling_station_id,trzaskowski_r1,nawrocki_r1,postal_code,trzaskowski_r2,nawrocki_r2,postal_clean,bucket,trzaskowski_k_score_1,nawrocki_k_score_1,trzaskowski_k_score_2,nawrocki_k_score_2,trzaskowski_flip,nawrocki_flip
12616,140706,1,105,285,26-910,467,193,26910,630,8.588235,-0.767123,10.044104,-10.044104,True,False
25866,261207,4,143,224,28-200,360,209,28200,662,2.443299,-1.491228,7.799246,-7.799246,True,False
5098,60903,4,89,174,23-100,260,163,23100,522,5.850746,-0.595588,5.824589,-5.824589,True,False
24825,260101,34,172,129,28-100,148,111,28100,657,-0.185567,-1.758333,-0.128350,0.128350,True,False
2372,40102,9,120,105,87-720,164,85,87720,2197,-0.204545,-1.139860,10.348880,-10.348880,True,False


In [28]:
# Anomalie na korzysc
df[cand_A + '_more_votes'] = df[cand_B + '_r2']<df[cand_B + '_r1']
df[cand_B + '_more_votes'] = df[cand_A + '_r2']<df[cand_A + '_r1']

To są rzeczywiście bardzo podejrzane przypadki i o takich przypadkach powinniśmy alarmować w pierwszej kolejności. Po pierwsze, ju na etapie wprowadzania do systemu, a po drugie do ewentualnej kontroli i ponownego liczenia glosów

## Wyniki

3.1. Identyfikacja komisji wyborczych z anomaliami

Z zastosowaniem konserwatywnej metody detekcji wartości odstających opartej na odchyleniu bezwzględnym od mediany (MAD) z progiem k > 3.0, zidentyfikowano 3 679 komisji wyborczych na poziomie obwodowym jako statystyczne anomalie. Stanowi to 11,6% wszystkich krajowych komisji wyborczych (z wyłączeniem zagranicznych i dyplomatycznych), których wyniki znacząco odbiegały od wyników sąsiednich komisji z tej samej lokalizacji geograficznej.
Przy nieco mniej rygorystycznym progu k > 2.0, liczba anormalnych komisji wzrosła do 5 453, co odpowiada 17,2% wszystkich komisji krajowych.


Analiza ujawniła także znaczne nagromadzenie przypadków, w których w jednej komisji występowały wiele typów anomalii jednocześnie. W szczególności:

* 1 482 komisje wykazywały co najmniej dwa typy anomalii,
* 216 komisji — co najmniej trzy typy,
* 33 komisje — wszystkie cztery typy anomalii.

In [29]:
# w zależności od wielkości k
k = 2

# deduplikacja, czyli jak wystąpiła więcej niz jedna anomalia, to liczymy taką komisję raz

### Na korzyść Nawrockiego

In [30]:
cand = "nawrocki"
opponent = "trzaskowski"
outliers_df = methods.generate_candidate_outliers(df, cand, opponent, k)

### Liczba występujących anomalii. Czyli w ilu komisjach wystąpiła np. anomalia "pop outlier"

In [31]:
bool_cols = ["pop_outlier", "growth_outlier", "flip", "more_votes"]
print(outliers_df[bool_cols].sum())

pop_outlier       3750
growth_outlier    3127
flip              1843
more_votes         128
dtype: int64


### Liczba komisji, w których wystąpiło minimum N anomalii, czyli minimum 1 (8108) albo wszystkie cztery anomalie (2)

In [32]:
true_counts = outliers_df[bool_cols].sum(axis=1)

# Convert to plain ints
count_distribution = {
    ">=1": int((true_counts >= 1).sum()),
    ">=2": int((true_counts >= 2).sum()),
    ">=3": int((true_counts >= 3).sum()),
    ">=4": int((true_counts >= 4).sum())
}

# Print row by row
for key, value in count_distribution.items():
    print(f"{key}: {value}")

>=1: 8108
>=2: 695
>=3: 43
>=4: 2


### Próba odwzorowania tabelki z wynikami

In [33]:
# suma głosów. jeżeli w komisji występuje więcej niż jedna anomalia, sumuj jej głosy tylko raz
outliers_df["anomalies"] = outliers_df.apply(methods.assign_top_anomaly, axis=1)

### Dodanie "brakujących głosów"

Głosy Nawrocki po / Głosy Trzaskowski po: Łączne liczby głosów po korekcie medianowej — czyli takie, jakie byłyby, gdyby udział Nawrockiego odpowiadał medianie w lokalnej grupie; głosy Trzaskowskiego dostosowano proporcjonalnie, przy zachowaniu łącznej liczby ważnych głosów w komisji.

In [34]:
outliers_df[cand + "_median_r2"] = outliers_df.groupby("bucket")[cand + "_r2"].transform("median")
outliers_df[opponent + "_median_r2"] = outliers_df.groupby("bucket")[opponent + "_r2"].transform("median")

In [35]:
outliers_df = methods.add_median_corrected_votes(outliers_df, cand, opponent)

In [36]:
summary_df = methods.summarize_by_anomaly(outliers_df, cand, opponent)
summary_df

,flaga,liczba,głosy nawrocki,głosy trzaskowski,różnica przed,głosy nawrocki po,głosy trzaskowski po,różnica po,zmiana
0,pop_outlier,3361,1851882,1537507,314375,1945077,1444312,500765,186390
1,growth_outlier,2814,752168,964366,-212198,831603,884931,-53328,158870
2,flip,1805,458674,369237,89437,377446,450465,-73019,-162456
3,more_votes,128,7028,3441,3587,5557,4912,645,-2942
4,łącznie,8108,3069752,2874551,195201,3159683,2784620,375063,179862


![kontek tabela 1 wyniki k=2](./../../images/kontek_tabela_wyniki_k2.png)

### Na korzyść Trzaskowskiego

In [37]:
cand = "trzaskowski"
opponent = "nawrocki"
k = 2
outliers_df = methods.generate_candidate_outliers(df, cand, opponent, k)

### Liczba występujących anomalii. Czyli w ilu komisjach wystąpiła np. anomalia "pop outlier"

In [38]:
bool_cols = ["pop_outlier", "growth_outlier", "flip", "more_votes"]
print(outliers_df[bool_cols].sum())

pop_outlier       4551
growth_outlier    3552
flip              2608
more_votes         112
dtype: int64


### Liczba komisji, w których wystąpiło minimum N anomalii, czyli minimum 1 (9487) albo wszystkie cztery anomalie (3)

In [39]:
true_counts = outliers_df[bool_cols].sum(axis=1)

# Convert to plain ints
count_distribution = {
    ">=1": int((true_counts >= 1).sum()),
    ">=2": int((true_counts >= 2).sum()),
    ">=3": int((true_counts >= 3).sum()),
    ">=4": int((true_counts >= 4).sum())
}

# Print row by row
for key, value in count_distribution.items():
    print(f"{key}: {value}")

>=1: 9487
>=2: 1288
>=3: 45
>=4: 3


In [40]:
outliers_df["anomalies"] = outliers_df.apply(methods.assign_top_anomaly, axis=1)

In [41]:
outliers_df[cand + "_median_r2"] = outliers_df.groupby("bucket")[cand + "_r2"].transform("median")
outliers_df[opponent + "_median_r2"] = outliers_df.groupby("bucket")[opponent + "_r2"].transform("median")

In [42]:
outliers_df = methods.add_median_corrected_votes(outliers_df, cand, opponent)

In [43]:
summary_df = methods.summarize_by_anomaly(outliers_df, cand, opponent)
summary_df

,flaga,liczba,głosy trzaskowski,głosy nawrocki,różnica przed,głosy trzaskowski po,głosy nawrocki po,różnica po,zmiana
0,pop_outlier,3495,1736016,1792115,-56099,1496470,2031661,-535191,-479092
1,growth_outlier,3303,539123,820984,-281861,593358,766749,-173391,108470
2,flip,2577,981128,785595,195533,778082,988641,-210559,-406092
3,more_votes,112,3528,2277,1251,2586,3219,-633,-1884
4,łącznie,9487,3259795,3400971,-141176,2870496,3790270,-919774,-778598


### PONOWNIE POLICZONE GŁOSY


https://polskieradio24.pl/artykul/3543223,jakie-sa-wyniki-w-komisjach-w-ktorych-ponownie-przeliczono-glosy-sprawdzilismy

In [44]:
df_vote_recount = presidential_data.load_data("2025", "2")
df_vote_recount = presidential_data.process_df(df_vote_recount, "2025", final_cols=["Gmina", "Województwo", "Siedziba"])

In [45]:
# Example: values from recount
target_nawrocki = 1132
target_trzaskowski = 540

# Find records that match these values exactly
matching_stations = df_vote_recount[
    (df_vote_recount["nawrocki"] == target_nawrocki) &
    (df_vote_recount["trzaskowski"] == target_trzaskowski)
]

print("Matching polling stations after recount:")
print(matching_stations.T)

Matching polling stations after recount:
                                                                11610
valid_votes                                                    1672.0
eligible_voters                                                1980.0
teryt_gmina                                                  126101.0
ballots_cast                                                   1684.0
polling_station_id                                                 95
nawrocki                                                       1132.0
trzaskowski                                                     540.0
postal_code                                                    31-346
Gmina                                                       m. Kraków
Województwo                                               małopolskie
Siedziba            Zespół Szkolno-Przedszkolny Nr 14, ul. Stawowa...


In [46]:
# --- Step 1: Create the recount dataset with both old and new values ---
recounts = [
    {"polling_station_id": 95,  "valid_votes": 1672, "old_nawrocki": 1132, "old_trzaskowski": 540,  "new_nawrocki": 540,  "new_trzaskowski": 1132},
    {"polling_station_id": 3,   "valid_votes": 1015, "old_nawrocki": 637,  "old_trzaskowski": 378,  "new_nawrocki": 377,  "new_trzaskowski": 638},
    {"polling_station_id": 13,  "valid_votes": 974,  "old_nawrocki": 611,  "old_trzaskowski": 363,  "new_nawrocki": 364,  "new_trzaskowski": 611},
    {"polling_station_id": 9,   "valid_votes": 639,  "old_nawrocki": 416,  "old_trzaskowski": 223,  "new_nawrocki": 223,  "new_trzaskowski": 416},
    {"polling_station_id": 25,  "valid_votes": 828,  "old_nawrocki": 504,  "old_trzaskowski": 324,  "new_nawrocki": 324,  "new_trzaskowski": 504},
    {"polling_station_id": 17,  "valid_votes": 931,  "old_nawrocki": 585,  "old_trzaskowski": 346,  "new_nawrocki": 344,  "new_trzaskowski": 585},
    {"polling_station_id": 30,  "valid_votes": 959,  "old_nawrocki": 610,  "old_trzaskowski": 349,  "new_nawrocki": 450,  "new_trzaskowski": 509},
    {"polling_station_id": 61,  "valid_votes": 1819, "old_nawrocki": 1048, "old_trzaskowski": 771,  "new_nawrocki": 771,  "new_trzaskowski": 1049},
    {"polling_station_id": 10,  "valid_votes": 330,  "old_nawrocki": 217,  "old_trzaskowski": 113,  "new_nawrocki": 317,  "new_trzaskowski": 363},
    {"polling_station_id": 53,  "valid_votes": 1458, "old_nawrocki": 628,  "old_trzaskowski": 830,  "new_nawrocki": 627,  "new_trzaskowski": 828},
    {"polling_station_id": 35,  "valid_votes": 928,  "old_nawrocki": 581,  "old_trzaskowski": 347,  "new_nawrocki": 347,  "new_trzaskowski": 581},
    {"polling_station_id": 6,   "valid_votes": 706,  "old_nawrocki": 368,  "old_trzaskowski": 338,  "new_nawrocki": 278,  "new_trzaskowski": 428},
    {"polling_station_id": 4,   "valid_votes": 797,  "old_nawrocki": 466,  "old_trzaskowski": 331,  "new_nawrocki": 331,  "new_trzaskowski": 466},
    {"polling_station_id": 4,   "valid_votes": 569,  "old_nawrocki": 209,  "old_trzaskowski": 360,  "new_nawrocki": 360,  "new_trzaskowski": 209},  # Staszów
    {"polling_station_id": 1,   "valid_votes": 660,  "old_nawrocki": 193,  "old_trzaskowski": 467,  "new_nawrocki": 468,  "new_trzaskowski": 192},  # Magnuszew
    {"polling_station_id": 113, "valid_votes": 1910, "old_nawrocki": 136,  "old_trzaskowski": 1774, "new_nawrocki": 296,  "new_trzaskowski": 1611},
    {"polling_station_id": 20,  "valid_votes": 1225, "old_nawrocki": 543,  "old_trzaskowski": 682,  "new_nawrocki": 542,  "new_trzaskowski": 683},
]

recount_df = pd.DataFrame(recounts)

# --- Step 2: Merge on 4 fields for exact match ---
df_affected_polling_stations = df_vote_recount.merge(
    recount_df,
    how="inner",
    left_on=["polling_station_id", "valid_votes", "nawrocki", "trzaskowski"],
    right_on=["polling_station_id", "valid_votes", "old_nawrocki", "old_trzaskowski"]
)

# --- Step 3: Output ---
# print("✅ Matches with recount corrections:")
df_affected_polling_stations[["teryt_gmina", "polling_station_id", "valid_votes", "nawrocki", "new_nawrocki",
               "trzaskowski", "new_trzaskowski"]].head(17)




,teryt_gmina,polling_station_id,valid_votes,nawrocki,new_nawrocki,trzaskowski,new_trzaskowski
0,20701.0,6,706.0,368.0,278,338.0,428
1,41804.0,4,797.0,466.0,331,331.0,466
2,46201.0,25,828.0,504.0,324,324.0,504
3,121611.0,10,330.0,217.0,317,113.0,363
4,126101.0,95,1672.0,1132.0,540,540.0,1132
5,140706.0,1,660.0,193.0,468,467.0,192
6,141201.0,13,974.0,611.0,364,363.0,611
7,146505.0,113,1910.0,136.0,296,1774.0,1611
8,160803.0,3,1015.0,637.0,377,378.0,638
9,161105.0,9,639.0,416.0,223,223.0,416


In [47]:
# Podsumowanie weryfikacji wyników wyborów 17 komisji

len(df_affected_polling_stations)

17

### Przepływy poparcia

Late Poll od Ipsos 
https://tvn24.pl/polska/wyniki-wyborow-prezydenckich-2025-jak-glosowali-polacy-interaktywne-grafiki-i-wykresy-st8486765


https://x.com/Jakub_Kubajek/status/1937434102312886412


https://colab.research.google.com/drive/1agO77d5cpwlEpMAzVwwT0xXQkmMinMyJ#scrollTo=rV1zoohwi7Xz

In [48]:
vote_flow_kn = {
    'BARTOSZEWICZ Artur': 0.696,
    'BIEJAT Magdalena Agnieszka': 0.117,
    'BRAUN Grzegorz Michał': 0.926,
    'HOŁOWNIA Szymon Franciszek': 0.146,
    'JAKUBIAK Marek': 0.895,
    'MACIAK Maciej': 0.707,
    'MENTZEN Sławomir Jerzy': 0.872,
    'NAWROCKI Karol Tadeusz': 0.993,
    'SENYSZYN Joanna': 0.193,
    'STANOWSKI Krzysztof Jakub': 0.521,
    'TRZASKOWSKI Rafał Kazimierz': 0.012,
    'WOCH Marek Marian': 0.538,
    'ZANDBERG Adrian Tadeusz': 0.165
}
new_votes_kn = 0.486

In [49]:
new_votes_rt = 0.514

vote_flow_rt = {key: 1 - value for key, value in vote_flow_kn.items()}

In [50]:
vote_flow_rt

{'BARTOSZEWICZ Artur': 0.30400000000000005,
 'BIEJAT Magdalena Agnieszka': 0.883,
 'BRAUN Grzegorz Michał': 0.07399999999999995,
 'HOŁOWNIA Szymon Franciszek': 0.854,
 'JAKUBIAK Marek': 0.10499999999999998,
 'MACIAK Maciej': 0.29300000000000004,
 'MENTZEN Sławomir Jerzy': 0.128,
 'NAWROCKI Karol Tadeusz': 0.007000000000000006,
 'SENYSZYN Joanna': 0.8069999999999999,
 'STANOWSKI Krzysztof Jakub': 0.479,
 'TRZASKOWSKI Rafał Kazimierz': 0.988,
 'WOCH Marek Marian': 0.46199999999999997,
 'ZANDBERG Adrian Tadeusz': 0.835}